This session motivated by [post](https://medium.com/stanford-cs224w/buy-this-session-based-recommendation-using-sr-gnn-d3415e393722) and how capture the each users' intention in the e-commerce system. so i planned this tutorial. i hope you guys this tutorial help to your research or application idea. :)

 # library

In [1]:
# Import Python built-in libraries
import copy
import pickle
import random
import time
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm, trange

from collections import Counter
import math
import os

# Import torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

# Import PyG packages
import torch_geometric as pyg
import torch_geometric.data as pyg_data
from torch_geometric.typing import Adj, OptTensor
import torch_sparse

/home/ubuntu/anaconda3/envs/graph/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


### 

pd.factorize()
torch.bincount()
np.argpartition()

Error

CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling cublasCreate(handle)
--> torch.nn.Embedding(embed , hidden_size) resize error


## pyg dataset tutorial

## pyg message function

# environment settings

In [2]:
# if you hope to get this dataset , visit this link http://www.recsyschallenge.com/2022/
os.chdir('./')

# dataset overview

[link](http://www.recsyschallenge.com/2022/)


- Sessions: The items that were viewed in a session. In this dataset a session is equal to a day, so a session is one user's activity on one day.


- Purchases: The purchase that happened at the end of the session. One purchased item per session.


- Item features: The label data of items. Things like “color: green,” “neckline: v-neck,” etc.

In [2]:
## original resource is converted the timestamp to unixtime because the session building.
## but, our dataset already has the session setup. we didn't that process
# timestamp to unixtime
candidate_item = pd.read_csv('candidate_items.csv')
item_df = pd.read_csv('item_features.csv')
train_df = pd.read_csv('train_sessions.csv')
train_label = pd.read_csv('train_purchases.csv')
# train['timestamp'] = pd.to_datetime(train['date'])
# train['unixtime'] = train['timestamp'].map(lambda x : time.mktime(x.timetuple()))
# train_label['timestamp'] = pd.to_datetime(train_label['date'])
# train_label['unixtime'] = train_label['timestamp'].map(lambda x : time.mktime(x.timetuple()))

# dataset explained

In [125]:
# 

def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

CounterA = Counter(item[item['item_id'] == 25357].sort_values('feature_category_id')['feature_value_id'])
CounterB = Counter(item[item['item_id'] == 21630].sort_values('feature_category_id')['feature_value_id'])

In [133]:
print(item['feature_category_id'].nunique())
print(item['feature_value_id'].nunique())
73 * 890

73
890


64970

In [112]:
item[item['item_id'] == 25357].sort_values('feature_category_id')

,item_id,feature_category_id,feature_value_id
425904,25357,3,793
425907,25357,4,618
425901,25357,5,58
425908,25357,7,452
425922,25357,15,133
425917,25357,17,378
425919,25357,19,112
425902,25357,26,126
425921,25357,32,352
425918,25357,34,275


In [113]:
item[item['item_id'] == 21630].sort_values('feature_category_id')

,item_id,feature_category_id,feature_value_id
363330,21630,3,793
363313,21630,4,618
363316,21630,5,605
363328,21630,7,452
363321,21630,15,133
363327,21630,17,378
363325,21630,19,765
363329,21630,26,126
363308,21630,32,902
363311,21630,41,145


# data check

In [3]:
sessionlist = list(set(train_df['session_id'].values))

session_ = np.random.choice(sessionlist, size=1)
print('view action')
display(train_df[train_df['session_id'] == session_[0]])
print('buy action')
display(train_label[train_label['session_id'] == session_[0]])

view action


,session_id,item_id,date
4464891,4179694,12969,2020-07-10 06:53:37.397


buy action


,session_id,item_id,date
941120,4179694,11060,2020-07-10 06:53:59.982


In [4]:
session_ = np.random.choice(sessionlist, size=1)

print('view action')
display(train_df[train_df['session_id'] == session_[0]]['item_id'].values)
view_action = train_df[train_df['session_id'] == session_[0]]['item_id'].values
print('buy action')
display(train_label[train_label['session_id'] == session_[0]]['item_id'].values)
buy_action = train_label[train_label['session_id'] == session_[0]]['item_id'].values

session_list = np.concatenate((view_action, buy_action)).tolist()
print(f'the session action list is {session_list}')

view action


array([ 2313, 19499,  4741, 16985, 14330, 21317])

buy action


array([5599])

the session action list is [2313, 19499, 4741, 16985, 14330, 21317, 5599]


# session setup

In [5]:
try:
    all_session=[]
    for sessions_ in tqdm(sessionlist):
        viewaction = train_df[train_df['session_id'] == sessions_]['item_id'].values
        #print(viewaction)
        buyaction = train_label[train_label['session_id'] == sessions_]['item_id'].values
        #print(buyaction)
        all_session.append(np.concatenate((viewaction, buyaction)).tolist())
        #print(all_session)
    np.save('all_session',all_session)
except:
    all_session = np.load('all_session.npy',allow_pickle=True)

In [6]:
for i in range(0,10):print(all_session[i])

[13596, 13596, 14392]
[9655, 9655, 15085]
[2814, 10096, 10096, 10373, 10096, 12251]
[5407, 15039, 6961, 17919]
[5154, 23282]
[24067, 4139, 9184]
[27173, 26160, 20412, 4179, 20412, 1107, 20412, 127, 22926, 22926, 25146, 18061, 24801]
[27349, 20495]
[24362, 11053, 26691, 11382]
[15654, 18626]


In [7]:
#example session format

print(f'train the sequence of the event {all_session[0][:-1]},and the action(buy) {[all_session[0][-1]]}')

train the sequence of the event [13596, 13596],and the action(buy) [14392]


In [277]:
# our datasets didn't have the visitors so, skipping this secton.

# def extract_subsessions(sessions):
#     '''Extracts all partial sessions from the sessions given.
    
#     (1,2,3) -> (1,2) and (1,2,3)
    
#     '''
#     all_sessions = []
#     for session in sessions:
#         for i in range(1, len(session)):
#             all_sessions.append(session[:i+1])
#     return all_sessions

# train test val splitting

In [8]:
sampling_rate = 1
seed = 42

num_train = int(len(all_session) * 0.8 * sampling_rate)
num_val = int(len(all_session) * 0.1 * sampling_rate)
num_test = int(len(all_session) * 0.1 * sampling_rate)

train_sessions = all_session[:num_train]
val_sessions = all_session[num_train : num_train+num_val]
test_sessions = all_session[num_train+num_val : num_train+num_val+num_test]

In [54]:
# Check the number of (sub)sessions in each split
print(f'train, val, and test sessions: {len(train_sessions), len(val_sessions), len(test_sessions)}')

directory = 'raw'
if os.path.isdir(directory)==False:
    os.makedirs(directory)
    
print(f'the {directory} made sucessfully.')
    
# Save the processed files.
with open('raw/train.txt', 'wb') as f:
    pickle.dump(train_sessions, f)
with open('raw/val.txt', 'wb') as f:
    pickle.dump(val_sessions, f)
with open('raw/test.txt', 'wb') as f:
    pickle.dump(test_sessions, f)

train, val, and test sessions: (800000, 100000, 100000)
the raw made sucessfully.


In [55]:
# load for sessions
path = 'raw/train.txt'
with open(path, 'rb') as f:
    pkl_data = pickle.load(f)

## dataset formatting for PyG

In [9]:
session_num = 123345
session, y = all_session[session_num][:-1], all_session[session_num][-1]
print(f'the session seqeunce(view action) is {session} \n\n')
print(f'the session result(buy action) is {[y]} \n\n')

codes, uniques = pd.factorize(session)

print(codes , '\n')
print(uniques)

the session seqeunce(view action) is [16462] 


the session result(buy action) is [12955] 


[0] 

[16462]


In [27]:
len(codes) , len(uniques)

(1, 1)

In [ ]:
senders, receivers = codes[:-1], codes[1:]

In [299]:
print(senders, '\n')
print(receivers)

[ 0  1  2  3  4  5  6  7  8  9  1 10 11] 

[ 1  2  3  4  5  6  7  8  9  1 10 11 12]


In [10]:
class GraphDataset(pyg_data.InMemoryDataset):
    def __init__(self, root, file_name, transform=None, pre_transform=None):
        self.file_name = file_name
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [f'{self.file_name}.txt']

    @property
    def processed_file_names(self):
        return [f'{self.file_name}.pt']

    def download(self):
        pass

    def process(self):
        raw_data_file = f'{self.raw_dir}/{self.raw_file_names[0]}'
        with open(raw_data_file, 'rb') as f:
            sessions = pickle.load(f)
        data_list = []

        for session in sessions:
            session, y = session[:-1], session[-1]
            codes, uniques = pd.factorize(session)
            senders, receivers = codes[:-1], codes[1:]

            # Build Data instance
            edge_index = torch.tensor([senders, receivers], dtype=torch.long)
            x = torch.tensor(uniques, dtype=torch.long).unsqueeze(1)
            y = torch.tensor([y], dtype=torch.long)
            data_list.append(pyg_data.Data(x=x, edge_index=edge_index, y=y))

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [11]:
class GatedSessionGraphConv(pyg.nn.conv.MessagePassing):
    # you can assign the aggregation function ex) sum , mean , add etc
    def __init__(self, out_channels, aggr: str ='add', **kwargs):
        super().__init__(aggr=aggr, **kwargs)
        
        self.out_channels = out_channels
        
        self.gru = torch.nn.GRUCell(out_channels, out_channels, bias=False)
    
    # edge index X node feature
    ## In this section , Done both aggregate and propagate work for neighbour information  fetching.
    def forward(self, x, edge_index):
        m = self.propagate(edge_index, x=x, size=None)
        x = self.gru(m , x)
        return x
    
    # source to target
    def message(self, x_j):
        return x_j
    
    # propagate and update
    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x, reduce=self.aggr)

# Train

In [13]:
# Define the hyperparameters.
# Code taken from 2021 Fall CS224W Colab assignments.
# num_items = max(item.item_id)
# num_items = train_label.item_id.nunique()
# num_items = len(set(candidate_item.item_id))
# num_items = item_df.item_id.nunique()
args = {
    'batch_size': 100,
    'hidden_dim': 32,
    'epochs': 100,
    'l2_penalty': 0.00001,
    'weight_decay': 0.1,
    'step': 30,
    'lr': 0.001,
    'num_items': 30000}

class objectview(object):
    def __init__(self, d): 
        self.__dict__ = d

args = objectview(args)

In [28]:
def train(args):
    # Prepare data pipeline
    train_dataset = GraphDataset('./', 'train')
    train_loader = pyg_data.DataLoader(train_dataset,
                                       batch_size=args.batch_size,
                                       shuffle=False,
                                       drop_last=True)
    val_dataset = GraphDataset('./', 'val')
    val_loader = pyg_data.DataLoader(val_dataset,
                                     batch_size=args.batch_size,
                                     shuffle=False,
                                     drop_last=True)

    # Build model
    model = SRGNN(args.hidden_dim, args.num_items).to('cuda')

    # Get training components
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=args.lr,
                                 weight_decay=args.l2_penalty)
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=args.step,
                                          gamma=args.weight_decay)
    criterion = nn.CrossEntropyLoss()

    # Train
    losses = []
    test_accs = []
    top_k_accs = []

    best_acc = 0
    best_model = None

    for epoch in range(args.epochs):
        total_loss = 0
        model.train()
        for _, batch in enumerate(tqdm(train_loader)):
            batch.to('cuda')
            optimizer.zero_grad()

            pred = model(batch)
            label = batch.y
            loss = criterion(pred, label)

            loss.backward()
            optimizer.step()
            total_loss += loss.item() * batch.num_graphs

        total_loss /= len(train_loader.dataset)
        losses.append(total_loss)

        scheduler.step()

        if epoch % 1 == 0:
          test_acc, top_k_acc = test(val_loader, model, is_validation=True)
          print(test_acc)
          test_accs.append(test_acc)
          top_k_accs.append(top_k_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])
  
    return test_accs, top_k_accs, losses, best_model, best_acc, val_loader

In [26]:
def test(loader, test_model, is_validation=False, save_model_preds=False):
    test_model.eval()

    # Define K for Hit@K metrics.
    k = 20
    correct = 0
    top_k_correct = 0

    for _, data in enumerate(tqdm(loader)):
        data.to('cuda')
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            score = test_model(data)
            pred = score.max(dim=1)[1]
            label = data.y

        if save_model_preds:
          data = {}
          data['pred'] = pred.view(-1).cpu().detach().numpy()
          data['label'] = label.view(-1).cpu().detach().numpy()

          df = pd.DataFrame(data=data)
          # Save locally as csv
          df.to_csv('pred.csv', sep=',', index=False)
            
        correct += pred.eq(label).sum().item()

        # We calculate Hit@K accuracy only at test time.
        if not is_validation:
            score = score.cpu().detach().numpy()
            for row in range(pred.size(0)):
                top_k_pred = np.argpartition(score[row], -k)[-k:]
                if label[row].item() in top_k_pred:
                    top_k_correct += 1
    
    if not is_validation:
        return correct / len(loader), top_k_correct / len(loader)
    else:
        return correct / len(loader), 0

In [22]:
class SRGNN(nn.Module):
    def __init__(self, hidden_size, n_items):
        super(SRGNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items
        
        # node features 
        self.embedding = nn.Embedding(self.n_items, self.hidden_size)
        self.gated = GatedSessionGraphConv(self.hidden_size)

        self.q = nn.Linear(self.hidden_size, 1)
        self.W_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.W_2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.W_3 = nn.Linear(2 * self.hidden_size, self.hidden_size, bias=False)

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, data):
        x, edge_index, batch_map = data.x, data.edge_index, data.batch
 
        #### TO DO LIST
        # you could feature update in this section.
        # not embed weight value (lookup) , using your own feature of node.
        # ex) the value of item , and derivation of feature.
        
        # (0)
        embedding = self.embedding(x).squeeze()

        # (1)-(5)
        v_i = self.gated(embedding, edge_index)

        # Divide nodes by session
        # For the detailed explanation of what is happening below, please refer
        # to the Medium blog post.
        sections = list(torch.bincount(batch_map).cpu())
        v_i_split = torch.split(v_i, sections)

        v_n, v_n_repeat = [], []
        for session in v_i_split:
            v_n.append(session[-1])
            v_n_repeat.append(
                session[-1].view(1, -1).repeat(session.shape[0], 1))
        v_n, v_n_repeat = torch.stack(v_n), torch.cat(v_n_repeat, dim=0)

        q1 = self.W_1(v_n_repeat)
        q2 = self.W_2(v_i)

        # (6)
        alpha = self.q(F.sigmoid(q1 + q2))
        s_g_split = torch.split(alpha * v_i, sections)

        s_g = []
        for session in s_g_split:
            s_g_session = torch.sum(session, dim=0)
            s_g.append(s_g_session)
        s_g = torch.stack(s_g)

        # (7)
        # global embedding
        ## the weighted average of the embeddigns of the items in the session.
        ### such as 'ATTENTION'
        s_l = v_n
        s_h = self.W_3(torch.cat([s_l, s_g], dim=-1))

        # (8) Unnormalized scroes 
        ## computing the cosine similairty between the session embedding (1 x d)
        ## if the embeddings of all 499867 items, then the un-normalized score will 
        ## (466867 x 1). This represents the scores for each item for the given session.
        z = torch.mm(self.embedding.weight, s_h.T).T
        return z

In [ ]:
test_accs, top_k_accs, losses, best_model, best_acc, test_loader = train(args) 

print(test_accs, top_k_accs)
print("Maximum test set accuracy: {0}".format(max(test_accs)))
print("Minimum loss: {0}".format(min(losses)))

# plt.title(dataset.name)
plt.plot(losses, label="training loss" + " - ")
plt.plot(test_accs, label="test accuracy" + " - ")
plt.legend()
plt.show()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 66.16it/s]


1.812


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 95.26it/s]


2.699


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 53.75it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 94.41it/s]


4.359


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 94.33it/s]


4.831


 54%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                         | 4341/8000 [01:43<01:22, 44.23it/s]

# evaluation

In [ ]:
torch.save(best_model.state_dict(), 'model')

In [ ]:
# Run test for our best model to save the predictions!
test_dataset = GraphDataset('./', 'test')
test_loader = pyg_data.DataLoader(test_dataset,
                                  batch_size=args.batch_size,
                                  shuffle=False,
                                  drop_last=True)

test(test_loader, best_model, is_validation=False, save_model_preds=True)

# code by jeongiitae6@gmail.com

if u have a question , give me the email above address. :)

and i special thanks to for allowing the reuse this nice quality resource Eunjee Lynn Sung [at] stanford.edu
